In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [0]:
# Create the agent

# 如前所述，此代理是无状态的。这意味着它不会记住以前的交互。为了给它内存，我们需要传入一个检查点。
# 在传入检查点时，我们还必须在调用代理时传入一个thread_id（以便它知道要从哪个线程/对话恢复）。
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
# 1. 定义工具
tools = [search]

# 2. 创建代理
# model_with_tools = model.bind_tools(tools)
# 请注意，我们传入的是模型，而不是model_with_tools。那是因为create_react_agent会在幕后为我们呼唤.bind_tools。
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# 3. 运行代理
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
var = response["messages"]

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
        {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
        {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")